In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
data = pd.read_csv('exams_dataset.csv')

### Вопрос 1: Можно ли предположить, что статистически количество мужчин и женщин, сдающих экзамены, неразличимо?

Нулевая гипотеза (H0): количество мужчин и женщин, сдающих экзамены, неразличимо. \
Альтернативная гипотеза (H1): Количество мужчин и женщин, сдающих экзамены, различимо.

Чтобы проверить эту гипотезу, используем критерий хи-квадрат на независимость:

In [3]:
contingency_table = pd.crosstab(data['gender'], columns='count')
res = stats.chisquare(contingency_table['count']) # categories are equally liked
print(f'{res.statistic=}')
print(f'{res.pvalue=}')

res.statistic=0.144
res.pvalue=0.7043364134884518


In [4]:
contingency_table

col_0,count
gender,
female,494
male,506


### Не отвергаем H0 => количество неразличимо

Статистический тест: критерий независимости Хи-квадрат.

Описание/пояснение: Критерий независимости хи-квадрат используется для определения наличия значимой связи между двумя категориальными переменными. В этом случае мы бы использовали его, чтобы проверить, существует ли существенная связь между полом и количеством людей, пишущих тест.

1. Статистика: Статистика Хи-квадрат.
2. Распределение при условии нулевой гипотезы: распределение Хи-квадрат с (r-1) x (c-1) степенями свободы, где r - количество строк, а c - количество столбцов в `contingency_table`
3. Тип критической области: двусторонняя критическая область.

### Вопрос 2: Тот же вопрос об этнической принадлежности.

Нулевая гипотеза (H0): количество людей разных национальностей, пишущих экзамены, неразличимо. \
Альтернативная гипотеза (H1): Количество людей разных национальностей, пишущих экзамены, различимо.

Выполняем еще один тест на независимость по хи-квадрату, на этот раз с переменной ```"race/ethnicity"```.

In [5]:
contingency_table = pd.crosstab(data["race/ethnicity"], columns='count')
res = stats.chisquare(contingency_table['count'])
print(f'{res.statistic=}')
print(f'{res.pvalue=}')

res.statistic=192.98999999999998
res.pvalue=1.2070807175456277e-40


In [6]:
contingency_table

col_0,count
race/ethnicity,
group A,77
group B,204
group C,324
group D,261
group E,134


### С уверенностью отвергаем H0 => количество различимо

Статистический тест: критерий Хи-квадрат на независимость (отдельно для каждого вопроса).

Описание/пояснение: Аналогично первому вопросу, но теперь применяется отдельно к каждому вопросу теста. Мы проверяем, существует ли существенная связь между этнической принадлежностью и количеством людей, пишущих каждый вопрос.

1. Статистика: Статистика Хи-квадрат (отдельно для каждого вопроса).
2. Распределение при условии нулевой гипотезы: распределение Хи-квадрат с (r-1) степенями свободы, где r - количество категорий для этнической принадлежности.
3. Тип критической области: двусторонняя критическая область.

### Вопрос 3: Проверьте гипотезу об однородности результатов по математике и письменной части (письму).

Нулевая гипотеза (H0): Результаты по математике и письму одинаковы. \
Альтернативная гипотеза (H1): Результаты по математике и письму неодинаковы.

Выполняем парный t-тест, чтобы сравнить результаты по математике и письму:

In [7]:
# Используем ttest_rel для зависимых
res = stats.ttest_rel(data['math score'], data['writing score'])

In [8]:
print(f'{res.statistic=}')
print(f'{res.pvalue=}')
print(f'{res.df=}')
print(f'interval: [{res.confidence_interval().low}, {res.confidence_interval().high}]')

res.statistic=-6.652585439515546
res.pvalue=4.736787232699879e-11
res.df=999
interval: [-2.5407394963580052, -1.3832605036419947]


### С уверенностью отвергаем H0 => результаты по математике и письму различимы

Статистический тест: t-тест с одной выборкой.

Описание/пояснение: t-критерий для одной выборки используется для сравнения среднего значения выборки с известным значением (в данном случае, средним значением равномерного распределения).

1. Статистика: t-статистика.
2. Распределение при условии нулевой гипотезы: t-распределение с (n - 1) степенями свободы, где n - размер выборки.
3. Тип критической области: Критическая область с двумя концами.

Этот подход предполагает, что нулевая гипотеза подразумевает равномерное распределение баллов, и мы проверяем, существенно ли наблюдаемое распределение отличается от этого ожидания.

## Второй тест Колмогорова-Смирнова для проверки однородности распределений

Сделан для проверки гипотезы о том, что две выборки из одного рапсределения \
Область двусторонняя

In [9]:
res = stats.ks_2samp(data['math score'], data['writing score'])
print(f'{res.statistic=}')
print(f'{res.pvalue=}')

res.statistic=0.061
res.pvalue=0.04839715079181246


### $pvalue < 0.05 \Rightarrow отвергаем$

### Вопрос 4: Правда ли, что те, кто закончил подготовительные курсы, лучше сдали экзамены?

Нулевая гипотеза (H0): Прохождение подготовительных курсов не влияет на успеваемость на экзамене. \
Альтернативная гипотеза (H1): Прохождение подготовительных курсов влияет на успеваемость на экзамене.

Выполняем t-критерий с двумя выборками, чтобы сравнить результаты тех, кто завершил курс, с теми, кто этого не сделал:

In [10]:
# Используем ttest_ind для независимых
completed = data[data['test preparation course'] == 'completed']['math score']
not_completed = data[data['test preparation course'] == 'none']['math score']
res = stats.ttest_ind(completed, not_completed)

In [11]:
print(f'{res.statistic=}')
print(f'{res.pvalue=}')
print(f'{res.df=}')
print(f'interval: [{res.confidence_interval().low}, {res.confidence_interval().high}]')

res.statistic=5.5589194118951095
res.pvalue=3.483493224877811e-08
res.df=998.0
interval: [3.768770670960493, 7.881362662372837]


### С уверенностью отвергаем H0 => курсы влияют на успеваемость

Статистический тест: t-критерий независимых выборок (при условии, что данные распределены приблизительно нормально).

Описание/пояснение: t-критерий независимых выборок используется для сравнения средних значений двух независимых групп, чтобы определить, есть ли между ними существенная разница.

1. Статистика: t-статистика.
2. Распределение при условии нулевой гипотезы: t-распределение с (n1 + n2 - 2) степенями свободы, где n1 и n2 - размеры выборки двух групп.
3. Тип критической области: Двухвостая критическая область.

## Используем U test Манна-Уитни

Сделан для малых непараметрических ($U$ not from $Norm(\mu, \sigma)$ / распределены в интервале) выборок \
Область двусторонняя

In [12]:
completed = data[data['test preparation course'] == 'completed']['math score']
not_completed = data[data['test preparation course'] == 'none']['math score']
res = stats.mannwhitneyu(completed, not_completed)

In [13]:
print(f'{res.statistic=}')
print(f'{res.pvalue=}')

res.statistic=140075.0
res.pvalue=2.2533705495929116e-07


## Отвергаем => курсы влияют

## Почему в 3 задании rel, а в 4 ind?

### ttest_ind (t-test независимых выборок):
Пример использования: Этот тест используется при сравнении средних значений двух независимых групп, чтобы определить, есть ли между ними статистически значимая разница.

Пример сценария: Если у вас есть две отдельные группы участников (например, группа, которая закончила подготовительные курсы, и группа, которая этого не сделала), и вы хотите проверить, есть ли существенная разница в их экзаменационных баллах.

### ttest_rel (t-test парных выборок):
Пример использования: Этот тест используется при сравнении средних значений двух связанных групп, часто когда для каждого измерения используются одни и те же испытуемые (разработка парных или повторяющихся измерений).

Пример сценария: Если у вас есть одна группа участников, и вы измеряете их результаты на экзаменах до и после завершения подготовительных курсов, вы можете использовать t-тест с парными выборками, чтобы определить, есть ли существенная разница в их результатах.

### Предположили что есть зависимость между баллами по разным предметам у одного человека!
### Можно было выдвинуть другой теорию, остановились на этой.

## Итого

## Разные стат тесты могут отвергать / не отвергать H0 на одной и той же задаче, тогда нужно проводить более глубокое статистическое исследование. Наши примеры были очевидны, поэтому тесты согласовывались.

Проверяем значения p для каждого теста. \
Если значение p меньше выбранного нами уровня значимости (например, 0,05), отвергаем нулевую гипотезу. Если значение p больше, тогда не можем отвергнуть нулевую гипотезу.